# Práctica 3 - CNN para clasificar imágenes de frutas - Lab 1
## Preparación de entorno
#### Instalar las librerías

In [7]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install torch
# %pip install torchvision

#### Importar librerías de código

In [8]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

## CNN SetUp
* **Red convolucional secuencial (CNN secuencial)** $\rightarrow$ Tipo de red neuronal convolucional diseñada utilizando un modelo secuenncial (una pila lineal de capas donde cada capa recibe la salida de la capa anterior como entrada).
  * **Convolución** $\rightarrow$ Operación matemática que combina dos funciones para producir una tercera función. En el contexto de las CNN, se utiliza para extraer características de las imágenes.
  * **Capas** $\rightarrow$ Capas de convolución, activación, pooling y completamente conectadas (fully connected).
  * Recibe **varias entradas** (como señales), cada una con un peso que indica su importancia.

* Componentes principales de una CNN secuencial:
  * **Capas convolucionales** $\rightarrow$ Aplican filtros para extraer características relevantes de las imágenes, como bordes, texturas o patrones. El tamaño y número de filtros son hiperparámetros que definimos nosotros.
  * **Funciones de activación** $\rightarrow$ Generalmente, se utiliza *ReLU* para introducir no linealidad en las capas intermedias, mientras que *Softmax* se utiliza en la capa de salida para problemas de clasificación multiclase.
  * **Capas de pooling** $\rightarrow$ Reducen las dimensiones espaciales (alto y ancho) de las características manteniendo las más relevantes. Esto disminuye el coste computacional y nos ayuda a prevenir el sobreajuste.
  * **Capas completamente conectadas** $\rightarrow$ Al final, las características que hemos extraído las aplanamos y las pasamos a una o más capas densas (completamente conectadas) para hacer la clasificación final.
  * **Optimizador y fución de pérdida** $\rightarrow$ Utilizamos *RMSprop* como optimizador y *categorical_crossentropy* como función de pérdida para problemas de clasificación multiclase. Estos son hiperparámetros que podemos ajustar según nuestras necesidades.

<img src="./media/Estructura cnn.jpg" width="70%" style="display: block; margin: 0 auto;"/>

<img src="./media/Arquitectura cnn.jpg" width="70%" style="display: block; margin: 0 auto; padding-top: 15px;"/>

* Aplicaciones de las redes convolucionales:
  * **Clasificación de imágenes** $\rightarrow$ Identificar objetos en imágenes, como en nuestro caso (clasificar frutas).
  * **Detección de objetos** $\rightarrow$ Localizar y clasificar múltiples objetos en una imagen.
  * **Segmentación de imágenes** $\rightarrow$ Dividir una imagen en regiones significativas, como identificar diferentes partes de una imagen médica.
  * **Reconocimiento facial** $\rightarrow$ Identificar y verificar caras en imágenes o videos.
  * **Procesamiento de video** $\rightarrow$ Analizar secuencias temporales como en coches autónomos o vigilancia.
  * **Generación de imágenes** $\rightarrow$ Crear imágenes nuevas a partir de datos existentes, como en el caso de *GANs* (Generative Adversarial Networks).
  * **Reconocimietnto de texto** $\rightarrow$ Extraer texto de imágenes (OCR).
  
### Implementación

Lo primero que vamos a hacer antes de entrenar la red, es preparar adecuadamente los datos. En este caso, vamos a utilizar el dataset de frutas [*"Fruits 360"*](https://www.kaggle.com/datasets/moltean/fruits/data), donde las imágenes están organizadas en carpetas según su clase (es decir, cada carpeta corresponde a una fruta distinta). Esto nos va a ayudar mucho, porque podemos utlizar la función *ImageFolder* de PyTorch, que automáticamente asigna una etiqueta a cada imagen según el nombre de la carpeta en la que se encuentra.

Como las redes neuronales no pueden procesar directamente JPEGs, necesitamos convertirlas a tensores, que son estructuras numéricas similares a matrices. Para ello vamos a usar una serie de transformaciones que aplicarmeros a cada imagen: primero las redimensionamos a un tamaño uniforme (64x64 píxeles), y luego las convertimos a tensores, lo cual también normaliza sus valores de píxel entre 0 y 1. Esto va ayudar a que nuestro modelo entrene de forma más estable.

> **Nota:** Cuanto más grande sea la imagen, más tiempo tardará en entrenar el modelo. En este caso, hemos elegido 64x64 píxeles como un tamaño intermedio que debería funcionar bien para nuestro problema. En general, es recomendable usar imágenes de tamaño uniforme para evitar problemas de memoria y mejorar la eficiencia del entrenamiento. Sin embargo, si tenemos imágenes de diferentes tamaños, podemos usar técnicas de *data augmentation* para aumentar la diversidad del dataset y mejorar la capacidad de generalización del modelo.

In [9]:
# Rutas de los directorios de datos
DIRECTORIO_ENTRENAMIENTO = './data/FandV/Training'
DIRECTORIO_PRUEBAS = './data/FandV/Test'

# Transformaciones que vamos a aplicar a las imágenes
transformacion = transforms.Compose([
    transforms.Resize((64, 64)),       # Redimensionamos todas las imágenes al mismo tamaño (64x64)
    transforms.ToTensor(),             # Convertimos la imagen a tensor (valores entre 0 y 1)
])

# Cargamos las imágenes usando ImageFolder, que usa la carpeta como etiqueta
dataset = datasets.ImageFolder(DIRECTORIO_ENTRENAMIENTO, transform=transformacion)

Una vez que ya hemos cagado todas las imágenes y las tenemos en el tensor, tenemos que dividir el conjunto de datos de `DIRECTORIO_ENTRENAMIENTO` en dos subconjuntos:
* **Conjunto de entrenamiento** $\rightarrow$ Lo vamos a utilizar para entrenar el modelo. Este conjunto tiene la mayoría de las imágenes y es donde el modelo aprende a reconocer patrones y características de las frutas.
* **Conjunto de validación** $\rightarrow$ Lo utilizamos para evaluar el rendimiento del modelo durante el entrenamiento. Este conjunto tiene menos imágenes y se utiliza para comprobar si el modelo está aprendiendo correctamente y no se está sobreajustando a los datos de entrenamiento.

Esta división nos va a permitir entrenar el modelo con una parte de los latos y luego comprobar como de bien generaliza con datos que no ha visto antes. Normalmente, se suele reservar un 20% de los datos para validación. Para dividir el dataset, vamos a utilizar la función `random_split` de PyTorch, que nos permite dividir un dataset en dos subconjuntos de forma aleatoria. Esta función toma como entrada el dataset original y las longitudes de los subconjuntos que queremos crear.

In [20]:
# Dividimos el dataset en entrenamiento y validacion (80%-20%)
tamano_entrenamiento = int(0.8 * len(dataset))
tamano_validacion = len(dataset) - tamano_entrenamiento

dataset_entrenamiento, dataset_validacion = random_split(dataset, [tamano_entrenamiento, tamano_validacion])

# Usamos DataLoader para cargar los datos en batches
# (batch_size=32 significa que cargamos 32 imágenes a la vez)
# (shuffle=True significa que mezclamos los datos en cada época)
loader_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=32, shuffle=True)
loader_validacion = DataLoader(dataset_validacion, batch_size=32, shuffle=False)

# Número de clases (etiquetas diferentes)
num_classes = len(dataset.classes)

# Mostramos la información del dataset
print("---------------------------------------------------------------------------------")
print("                             Información del dataset                             ")
print("---------------------------------------------------------------------------------")
print(f"* Directorio de entrenamiento: {DIRECTORIO_ENTRENAMIENTO}")
print(f"* Directorio de pruebas: {DIRECTORIO_PRUEBAS}")
print("\n")
print(f"* Número total de imágenes: {len(dataset)}")
print(f"* Número de imágenes de entrenamiento: {len(dataset_entrenamiento)}")
print(f"* Número de imágenes de validación: {len(dataset_validacion)}")
print(f"* Dimensiones de las imágenes: {dataset[0][0].shape}")
print("\n")
print(f"* Número de batches de entrenamiento: {len(loader_entrenamiento)}")
print(f"* Número de batches de validación: {len(loader_validacion)}")
print(f"* Tamaño de cada batch de entrenamiento: {loader_entrenamiento.batch_size}")
print(f"* Tamaño de cada batch de validación: {loader_validacion.batch_size}")
print("\n")
print(f"* Número de clases: {num_classes}")
print(f"* Ejemplo de clases: {dataset.classes[30:34]}")

---------------------------------------------------------------------------------
                             Información del dataset                             
---------------------------------------------------------------------------------
* Directorio de entrenamiento: ./data/FandV/Training
* Directorio de pruebas: ./data/FandV/Test


* Número total de imágenes: 92545
* Número de imágenes de entrenamiento: 74036
* Número de imágenes de validación: 18509
* Dimensiones de las imágenes: torch.Size([3, 64, 64])


* Número de batches de entrenamiento: 2314
* Número de batches de validación: 579
* Tamaño de cada batch de entrenamiento: 32
* Tamaño de cada batch de validación: 32


* Número de clases: 180
* Ejemplo de clases: ['Apricot 1', 'Avocado 1', 'Avocado ripe 1', 'Banana 1']


### Referencias
* [Convolutional neural network - ScienceDirect](https://www.sciencedirect.com/topics/computer-science/convolutional-neural-network).